In [21]:
import cobra
from cobra.io import load_json_model, save_json_model
from cobra import Model, Reaction, Metabolite
import numpy as np
from math import exp
import pandas as pd
from matplotlib import pyplot as plt
from fba_functions import *
import seaborn as sns
from scipy import stats
import pickle

In [22]:
def load_pickle(filename):
    temp = None
    with open(filename,'rb') as f:
        temp = pickle.load(f)
    return temp
def dump_pickle(file, filename):
    with open(filename, 'wb') as f:
        pickle.dump( file , f)

In [2]:
ipsc_core = load_json_model('../data/gems/iPSC_core.json')
Adata = pd.read_csv('../data/gems/EnzymeActivity.csv')
A_dict = {Adata['Rxn'][i]:Adata['Activity(mmol/hr/g)'][i] for i in range(len(Adata.index))}
def get_grs(biomass, time_pts):
    output = []
    for i in range(len(biomass)-1):
        output.append( np.log(biomass[i+1]/biomass[i]) /(time_pts[i+1]-time_pts[i]) )
    return output  
def avg_cons(cons):
    output = []
    for i in range(len(cons)-1):
        output.append( (cons[i+1]+cons[i])/2 )
    return output
def norm_rates(biomass, cons, time_pts):
    output = []
    for i in range(len(biomass)-1):
        output.append( ((cons[i+1]-cons[i])/(biomass[i]/2+biomass[i+1]/2) )/(time_pts[i+1]-time_pts[i]) )
    return output
def get_pcc(x,y):
    coeff, pv = stats.pearsonr( x, y)
    return coeff

In [3]:
data_low=pd.read_csv('../data/exp_data/CultureData_lowNH4.csv')
data_low=data_low.rename(columns={'Biomass_avg(g/L)':'Biomass_avg','Biomass_std(g/L)':'Biomass_std'})
data_high=pd.read_csv('../data/exp_data/CultureData_highNH4.csv')
data_high=data_high.rename(columns={'Biomass_avg(g/L)':'Biomass_avg','Biomass_std(g/L)':'Biomass_std'})
mets=['Glucose','Gln','Lac','NH4']
Ts= [data_low.Time[i]/2+data_low.Time[i+1]/2 for i in range(len(data_low.Time)-1)]
grs=get_grs(data_low.Biomass_avg, data_low.Time)
approx_low = pd.DataFrame(zip(Ts,grs),columns=['Time','gr'])
for met in mets:
    approx_low[met+'_rate']=norm_rates( data_low.Biomass_avg, data_low[met+'_avg'], data_low.Time)
    approx_low[met+'_conc']=avg_cons( data_low[met+'_avg'] )
Ts=[data_high.Time[i]/2+data_high.Time[i+1]/2 for i in range(len(data_high.Time)-1)]
grs=get_grs(data_high.Biomass_avg, data_high.Time)
approx_high = pd.DataFrame(zip(Ts,grs),columns=['Time','gr'])
for met in mets:
    approx_high[met+'_rate']=norm_rates( data_high.Biomass_avg, data_high[met+'_avg'], data_high.Time)
    approx_high[met+'_conc']=avg_cons( data_high[met+'_avg'] )

# 1. Low initial NH4+

In [23]:
ptot = 0.67366; NGAM = 2.2771;T=60;#start from T=42hr
ic_lowN={'BIOMASS':0.155,'nh4_e':2.320,'lac__L_e':9.233,'glc__D_e':11.567,'gln__L_e':3.333}

In [24]:
# met_profile_lowN, flux_profile_lowN = dpcfba(ipsc_core, ptot, NGAM, A_dict, ic_lowN, T)
# met_profile_lowN['T'] = [t+42 for t in met_profile_lowN['T']]

In [45]:
# dump_pickle(met_profile_lowN, '../data/simulation_results/met_profile_lowN.pkl')
# dump_pickle(flux_profile_lowN, '../data/simulation_results/flux_profile_lowN.pkl')

In [46]:
met_profile_lowN=load_pickle('../data/simulation_results/met_profile_lowN.pkl')
flux_profile_lowN=load_pickle('../data/simulation_results/flux_profile_lowN.pkl')

# 2. High initial NH4+

In [51]:
ptot = 0.67366; NGAM = 2.2771;T=70; #start from T=30hr
ic_highN={'BIOMASS':0.047,'nh4_e':4.23,'lac__L_e':2.533,'glc__D_e':15.133,'gln__L_e':4.753}

In [52]:
# met_profile_highN, flux_profile_highN = dpcfba(ipsc_core, ptot, NGAM, A_dict, ic_highN, T)
# met_profile_highN['T'] = [t+30 for t in met_profile_highN['T']]

In [60]:
# dump_pickle(met_profile_highN, '../data/simulation_results/met_profile_highN.pkl')
# dump_pickle(flux_profile_highN, '../data/simulation_results/flux_profile_highN.pkl')

In [61]:
met_profile_highN=load_pickle('../data/simulation_results/met_profile_highN.pkl')
flux_profile_highN=load_pickle('../data/simulation_results/flux_profile_highN.pkl')